In [1]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [2]:
pip install PyPDF2

In [5]:
import re
from PyPDF2 import PdfReader
import json
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

#Vad är TTJ?
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def clean_text(text):
    # Decode encoded characters
    text = text.encode('latin1', errors='ignore').decode('unicode_escape')

    # Remove metadata and non-text content
    text = re.sub(r'\d+\s+obj\s+<<.*?>>\s+endobj', '', text, flags=re.DOTALL)

    # Handle special characters
    # Add more replacements as needed
    text = text.replace('\n', ' ').replace('\r', '')

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# Initialize the BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('KB/bert-base-swedish-cased')
tokenizer = BertTokenizer.from_pretrained('KB/bert-base-swedish-cased')

def generate_answer(user_question, text_content):
    inputs = tokenizer.encode_plus(user_question, text_content, add_special_tokens=True, return_tensors="pt", max_length=512, truncation='only_second')

    # Get the output from the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('swedish')) # Assuming the text is in Swedish
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

def create_json(question, answer, context):
    return {
        "Fråga": question,
        "Svar": answer,
        "Kontext": context
    }

pdf_path = 'TDOK_2015-0430.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(extracted_text)

print("Extracted and cleaned text:")
print(cleaned_text)

# Preprocess the text
preprocessed_text = preprocess_text(cleaned_text)
print("Preprocessed text:")
print(preprocessed_text)

print("Hej! Jag är en chatbot som kan hjälpa dig med information från en textfil. Fråga mig vad som helst (eller skriv 'avsluta' för att sluta).")

while True:
    user_question = input("Ange din fråga: ")

    if user_question.lower() == 'avsluta':
        print("Tack för att du chattade med mig. Adjö!")
        break

    genererat_svar = generate_answer(user_question, preprocessed_text)
    print("Svar:", genererat_svar)

    # Create JSON data (optional)
    json_data = create_json(user_question, genererat_svar, cleaned_text)
    print("JSON Data:")
    print(json_data)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracted and cleaned text:
TRAFIKSÄKERHETSBESTÄMMELSE1 (8) TDOK-nummer Dokumentdatum Version TDOK 2015:0430 2020-02-10 2.0 Fastställt av Gäller från Ersätter Chef PLkvtj [Gäller från] [Ersätter] Skapat av Konfidentialitetsnivå Cronvall Arne, UHtu Ej begränsadTMALL 0436 Trafiksäkerhetsbestämmelse 2.0Undantag vid provkörning i Trafikverkets spåranläggning Detta dokument ingår i Trafikverkets ledningssystem och är en del av säkerhetsstyrningssystemet för järnväg. Se särskilda regler för förvaltning av säkerhetstillstånd Innehåll 1 Syfte.........................................................................................................................................2 2 Omfattning ..............................................................................................................................2 3 Definitioner och förkortningar ................................................................................................2 4 Bestämmelser om undantag ......................